# Setup

Import Semantic Kernel SDK from pypi.org

In [35]:
%pip install -U semantic-kernel --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from semantic_kernel import __version__

__version__

'1.22.0'

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

>NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.

#### Using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

Let's define our kernel for this example.

In [5]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

We now configure our Chat Completion service on the kernel.

In [6]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    service_id="default"
)

Let's try chat with LLM model.

In [7]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = AzureChatPromptExecutionSettings()

system_message = """
You are a helpful assistant that can answer questions and provide information.
"""

chat_history = ChatHistory(system_message=system_message)

chat_history.add_user_message("Hello, how are you?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

print(response)

Hello! I'm here and ready to help you with any questions or information you need. How can I assist you today?


In [8]:
from semantic_kernel import Kernel

kernel = Kernel()

kernel.add_service(chat_completion)



Creating a plugin in Semantic Kernel.

In [13]:
from semantic_kernel.functions import kernel_function

class LightsPlugin:

    lights = [
        {"id": 1, "name": "Table Lamp", "is_on": False},
        {"id": 2, "name": "Porch light", "is_on": False},
        {"id": 3, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(name="get_lights", description="Gets a list of lights and their current state",)
    def get_state(self) -> str:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(name="change_state", description="Changes the state of the light")
    def change_state(self, id: int, is_on: bool) -> str:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None

In [15]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings

# Add a plugin (the LightsPlugin class is defined above)
kernel.add_plugin(LightsPlugin(), plugin_name="Lights")

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()

history.add_user_message("turn on all lights")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Print the results
print("Assistant > " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)

Assistant > All the lights are already turned on. If you need any further assistance, feel free to ask!


In [16]:
history.add_user_message("turn off table lights")

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Print the results
print("Assistant > " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)

Assistant > The Table Lamp has been turned off. If you need any more help, let me know!
